In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key=''
)

response = llm.invoke('Who is Vishnu?')
print(response.content)

Vishnu is a major deity in Hinduism and is considered one of the three primary aspects of the Supreme Being, along with Brahma and Shiva. He is often referred to as the Preserver or the Protector of the universe.

In Hindu mythology, Vishnu is depicted as a blue-skinned god with four arms, holding a conch shell, a discus, a mace, and a lotus flower. He is often shown reclining on a serpent named Shesha, who represents the cycles of time.

Vishnu is believed to be the embodiment of goodness and righteousness, and is often associated with the concepts of truth, justice, and morality. He is said to have taken various avatars (incarnations) to restore balance and order in the universe, and to protect his devotees from evil.

Some of the most well-known avatars of Vishnu include:

1. Matsya (the fish avatar): Vishnu took the form of a fish to save the world from a great flood.
2. Kurma (the tortoise avatar): Vishnu took the form of a tortoise to help the gods churn the ocean of milk and obt

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://careers.ltimindtree.com/job/Bengaluru-Senior-Software-Engineer/679605001/')
page_data = loader.load().pop().page_content
print(page_data)













Senior Software Engineer Job Details | LTIMindtree

























Press Tab to Move to Skip to Content Link
Skip to main content








Home
Search Jobs
Company
Join Talent Community




















Search by Keyword




Search by Location







                                 
                            





Clear












Home
Search Jobs
Company
Join Talent Community









View Profile





















Search by Keyword




Search by Location






Show More Options





Loading...







                                            Country/Region
                                        


All





                                            Location
                                        


All









                                 
                            





Clear
















Select how often (in days) to receive an alert:



 Create Alert







×



Select how often (in days) to receive an alert:














Apply now »



In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTION:
    The scraped text is from career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE) 
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Software Engineer",
  "experience": "2 to 4 years",
  "skills": [
    "Ruby software development",
    "Domain Specific Language (DSL) concept",
    "Relational Databases, especially Oracle"
  ],
  "description": "Understand the current capabilities and functionalities of the existing production platform, Deliver new functionality for the internal production platform for the ESG Business, Datapoint and Calculation development for ESG Business, Analyze requirements, recommend and implement solutions, and assist UAT"
}
```


In [10]:
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_result = json_parser.parse(res.content)
json_result

{'role': 'Senior Software Engineer',
 'experience': '2 to 4 years',
 'skills': ['Ruby software development',
  'Domain Specific Language (DSL) concept',
  'Relational Databases, especially Oracle'],
 'description': 'Understand the current capabilities and functionalities of the existing production platform, Deliver new functionality for the internal production platform for the ESG Business, Datapoint and Calculation development for ESG Business, Analyze requirements, recommend and implement solutions, and assist UAT'}

In [12]:
type(res.content)

str

In [13]:
import pandas as pd 
df = pd.read_csv('techstack_links.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import chromadb, uuid

client = chromadb.PersistentClient('vectorstore')
'''
chromadb.PersistentClient: saves the data in disk to use it later
chromadb.Client: saves the data in memory for temporary use
'''
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())]) # uuid = universally unique identifier, uuid.uuid4(): Generates a random UUID based on random numbers.

In [17]:
links = collection.query(query_texts=['Experience in python', 'expertise in react'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [19]:
job = json_result
job['skills']

['Ruby software development',
 'Domain Specific Language (DSL) concept',
 'Relational Databases, especially Oracle']

In [20]:
prompt_email = PromptTemplate.from_template(
    """ 
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Vishnu, a business development executive at VTG. VTG is an AI & Software Consulting company.
    Your job is to write a cold email to the client regarding the job mentioned above describing the role and fulfilling their needs.
    Also add the most relevat ones from the following links to showcase VTG's portfolio: {link_list}
    Remember you are Vishnu, BDE at VTG.
    ### EMAIL (NO PREAMBLE):
     
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Senior Software Engineer for ESG Business Platform Development

Dear Hiring Manager,

I hope this email finds you well. I came across the job description for a Senior Software Engineer at your esteemed organization, and I am excited to introduce VTG, an AI & Software Consulting company, as a potential partner to fulfill your requirements.

The role of a Senior Software Engineer at your organization involves understanding the current capabilities and functionalities of the existing production platform, delivering new functionality for the internal production platform for the ESG Business, and developing datapoints and calculations for the ESG Business. Our team of expert software engineers, with 2 to 4 years of experience in Ruby software development, Domain Specific Language (DSL) concept, and Relational Databases (especially Oracle), can seamlessly integrate with your team to analyze requirements, recommend and implement solutions, and assist in UAT.

At VTG, we have a